In [1]:
# If missing requirements: !pip install geopandas opera-utils pyarrow seaborn

In [2]:
import geopandas as gpd
import opera_utils
import pandas as pd

In [3]:
df = pd.read_parquet("disp-s1-unwrapping-runtimes.parquet")

# Convert to hours
runtime_cols = [
    "runtime_wrapped_phase",
    "runtime_stitching_bursts",
    "runtime_unwrap",
    "runtime_timeseries_inversion",
    "runtime_dolphin",
    "runtime_product",
    "runtime_disp_s1",
]
for col in runtime_cols:
    df[col] = (df[col] / 3600)
# Each ministack has 42 ifgs (except for first, which does 39)
# We run a default of n_parallel_jobs = 4 , factor back for the per-ifg number
# https://github.com/opera-adt/disp-s1/blob/d6e27cc06bbeda8ca1c3a0a7c98b351597e1d825/configs/algorithm_parameters_historical_20250401.yaml#L87
df["runtime_average_per_ifg_minutes"] = df["runtime_unwrap"] * 60 / 42 * 4


# Add the DISP-S1 Frame geometries
gdf = gpd.GeoDataFrame(
    pd.merge(
        df, opera_utils.get_frame_geodataframe(), left_on="frame_id", right_index=True
    )
)

In [4]:
gdf.head(2)

,frame_id,product_version,generation_datetime,burst_id_count,unwrap_method,max_memory_gb,venue,logname,runtime_wrapped_phase,runtime_stitching_bursts,...,spurt_num_points_per_tile,spurt_fraction_good,dolphin_version,disp_s1_version,log_path,runtime_average_per_ifg_minutes,is_land,is_north_america,orbit_pass,geometry
0,8882,0.8,2024-11-14 23:56:30,27,snaphu,34.18,pst,OPERA_L3_DISP-S1_IW_F08882_v0.8_20241114T23563...,NaN,NaN,...,None,None,0.33.0,0.5.1,s3://opera-pst-rs-pop1/products/DISP_S1/OPERA_...,NaN,1,True,ASCENDING,"POLYGON ((-97.38477 30.02259, -94.83005 30.547..."
1,8622,0.8,2024-11-14 23:59:18,25,snaphu,35.11,pst,OPERA_L3_DISP-S1_IW_F08622_v0.8_20241114T23591...,NaN,NaN,...,None,None,0.33.0,0.5.1,s3://opera-pst-rs-pop1/products/DISP_S1/OPERA_...,NaN,1,True,ASCENDING,"POLYGON ((-75.36963 41.30259, -72.41804 41.813..."


In [5]:
def explore_column(  # noqa: D103
    gdf, col_name: str, agg_func="mean", cmap="RdYlBu_r", vmin_pct=.20, vmax_pct=0.95
) -> gpd.GeoDataFrame:
    out = gdf.groupby("frame_id")[["geometry", col_name]].agg(
        {"geometry": "first", col_name: agg_func}
    )

    out = out.set_geometry("geometry").set_crs(gdf.crs)
    vmin = out[col_name].quantile(vmin_pct)
    vmax = out[col_name].quantile(vmax_pct)
    return out.explore(column=col_name, cmap=cmap, vmin=vmin, vmax=vmax)

In [6]:
gdf_snaphu_ops = gdf[
    (gdf.unwrap_method == "snaphu") & (gdf.venue == "ops")
]

In [7]:
gdf_snaphu_ops.describe()

,frame_id,product_version,generation_datetime,burst_id_count,max_memory_gb,runtime_wrapped_phase,runtime_stitching_bursts,runtime_unwrap,runtime_timeseries_inversion,runtime_dolphin,runtime_product,runtime_disp_s1,spurt_total_num_points,runtime_average_per_ifg_minutes,is_land
count,5645.000000,5645.0,5645,5645.000000,5645.000000,5645.000000,5645.000000,5645.000000,5645.000000,5645.000000,5645.000000,5645.000000,0.0,5645.000000,5645.0
mean,23123.022675,1.0,2025-06-17 01:48:24.282373,21.344730,36.126062,0.304936,0.131395,4.937895,0.186661,6.100450,0.556183,6.656634,NaN,28.216541,1.0
min,831.000000,1.0,2025-04-13 17:37:06,1.000000,29.570000,0.090339,0.040833,0.906667,0.078056,1.219119,0.245281,1.488789,NaN,5.180952,1.0
25%,12639.000000,1.0,2025-04-18 11:15:34,16.000000,34.390000,0.274822,0.112311,4.439414,0.170278,5.500436,0.475900,5.995500,NaN,25.368079,1.0
50%,22668.000000,1.0,2025-07-25 11:39:29,27.000000,36.060000,0.318997,0.145775,5.053306,0.181111,6.382514,0.598867,7.000714,NaN,28.876032,1.0
75%,34223.000000,1.0,2025-07-28 09:15:05,27.000000,37.600000,0.346792,0.156006,5.676122,0.210278,7.032439,0.638725,7.657431,NaN,32.434984,1.0
max,46293.000000,1.0,2025-08-11 12:03:17,27.000000,45.490000,0.455994,0.177694,10.575800,0.262222,12.006244,0.742028,12.685083,NaN,60.433143,1.0
std,13197.558690,0.0,NaN,7.978345,2.205721,0.059655,0.033708,1.323804,0.027651,1.597705,0.110720,1.687554,NaN,7.564593,0.0


In [8]:
explore_column(
    gdf_snaphu_ops,
    "runtime_disp_s1",
    vmax_pct=.95,
    agg_func='mean',
)

In [9]:
explore_column(
    gdf_snaphu_ops,
    "runtime_wrapped_phase",
)

In [10]:
explore_column(
    gdf_snaphu_ops,
    "runtime_unwrap",
)

In [11]:
explore_column(
    gdf_snaphu_ops,
    "runtime_average_per_ifg_minutes",
    agg_func="median",
)